# Capstone 3 - Book Recommendation System

# Pre Processing and Training Data Development

Pre Processing and Training Data Development is the fourth step in the Data Science Method. The following will be performed in this step:

1. Create dummy or indicator features for categorical variables
2. Standardize the magnitude of numeric features
3. Split into testing and training datasets
4. Apply scaler to the testing set

# Modeling

Modeling is the fifth step in the Data Science Method.  The following will be performed in this step:

1. Fit Models with Training Data Set
2. Review Model Outcomes — Iterate over additional models as needed.
3. Identify the Final Model

In [1]:
#load python packages
import os
import pandas as pd
import pandas.api.types as ptypes
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import warnings 
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("../Data_Wrangle_EDA/data/Cap3_step23_output.csv")
df.head()

,Unnamed: 0,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,User-ID,Book-Rating,Location,Age
0,13719,0452282152,Girl with a Pearl Earring,Tracy Chevalier,2001,Plume Books,85526.0,7.0,"victoria, british columbia, canada",36.0
1,13722,0380012863,Jonathan Livingston Seagull,Richard Bach,1976,Avon,85526.0,9.0,"victoria, british columbia, canada",36.0
2,13724,0375726403,Empire Falls,Richard Russo,2002,Vintage Books USA,85526.0,9.0,"victoria, british columbia, canada",36.0
3,13725,0312195516,The Red Tent (Bestselling Backlist),Anita Diamant,1998,Picador USA,85526.0,8.0,"victoria, british columbia, canada",36.0
4,13726,0385721420,Three Junes,JULIA GLASS,2003,Anchor,85526.0,7.0,"victoria, british columbia, canada",36.0


In [3]:
df = df.drop(["Unnamed: 0"], axis=1)
df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,User-ID,Book-Rating,Location,Age
0,0452282152,Girl with a Pearl Earring,Tracy Chevalier,2001,Plume Books,85526.0,7.0,"victoria, british columbia, canada",36.0
1,0380012863,Jonathan Livingston Seagull,Richard Bach,1976,Avon,85526.0,9.0,"victoria, british columbia, canada",36.0
2,0375726403,Empire Falls,Richard Russo,2002,Vintage Books USA,85526.0,9.0,"victoria, british columbia, canada",36.0
3,0312195516,The Red Tent (Bestselling Backlist),Anita Diamant,1998,Picador USA,85526.0,8.0,"victoria, british columbia, canada",36.0
4,0385721420,Three Junes,JULIA GLASS,2003,Anchor,85526.0,7.0,"victoria, british columbia, canada",36.0


In [4]:
df.shape

(88793, 9)

In [5]:
df.tail()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,User-ID,Book-Rating,Location,Age
88788,0373244738,The Prince &amp; The Pregnant Princess (Deser...,Susan Mallery,2002,Silhouette,163759.0,8.0,"abertillery, wales, united kingdom",37.0
88789,0373243707,Unexpectedly Expecting ! (Lone Star Canyon) (S...,Susan Mallery,2001,Silhouette,163759.0,7.0,"abertillery, wales, united kingdom",37.0
88790,0373244533,THE SHEIK &amp; THE VIRGIN PRINCESS - DESERT R...,Susan Mallery,2002,Silhouette,163759.0,8.0,"abertillery, wales, united kingdom",37.0
88791,037324200X,"Older, Wiser ... Pregnant (Special Edition , N...",Marilyn Pappano,1998,Silhouette,163759.0,8.0,"abertillery, wales, united kingdom",37.0
88792,0373224494,Forget Me Not (Rocky Mountain Rescue) (Harlequ...,Cassie Miles,1997,Harlequin,163759.0,7.0,"abertillery, wales, united kingdom",37.0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88793 entries, 0 to 88792
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ISBN                 88793 non-null  object 
 1   Book-Title           88793 non-null  object 
 2   Book-Author          88793 non-null  object 
 3   Year-Of-Publication  88793 non-null  int64  
 4   Publisher            88793 non-null  object 
 5   User-ID              88793 non-null  float64
 6   Book-Rating          88793 non-null  float64
 7   Location             88793 non-null  object 
 8   Age                  88793 non-null  float64
dtypes: float64(3), int64(1), object(5)
memory usage: 6.1+ MB


In [7]:
df_reviews = df[['User-ID','ISBN','Book-Rating']]
df_reviews.head()

,User-ID,ISBN,Book-Rating
0,85526.0,0452282152,7.0
1,85526.0,0380012863,9.0
2,85526.0,0375726403,9.0
3,85526.0,0312195516,8.0
4,85526.0,0385721420,7.0


In [8]:
df_reviews.shape

(88793, 3)

# This size is too big for my computer memory.  Using sample size of 5000.

In [9]:
df_sample = df_reviews.sample(n=5000, random_state=1)
df_sample.head()

,User-ID,ISBN,Book-Rating
24466,56399.0,0880291540,10.0
15963,204864.0,0385509510,10.0
65040,60707.0,0061092606,7.0
65546,123056.0,0446611859,8.0
5497,47971.0,0818405422,8.0


# Using scikit-surprise

In [15]:
from sklearn.model_selection import train_test_split

trainset, testset = train_test_split(df_sample, test_size=0.25, random_state=1)

In [18]:
from surprise import KNNWithMeans
my_k = 15
my_min_k = 5
my_sim_option = {
    'name':'pearson', 'user_based':True, 
    }
algo = KNNWithMeans(
    k = my_k, min_k = my_min_k, sim_option = my_sim_option
    )
algo.fit(trainset)

AttributeError: 'DataFrame' object has no attribute 'n_users'

# Using sklearn-recommender

In [19]:
import sklearn_recommender as skr

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Umesh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Umesh\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Umesh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Umesh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\Umesh\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Umesh\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-

# Transformer

In [20]:
tf = skr.transformer.UserItemTransformer(user_col='User-ID', item_col='ISBN', value_col='Book-Rating', agg_fct='mean')
user_item = tf.transform(df_sample)

In [21]:
user_item.head()

ISBN,0002553384,0004704746,0004720695,0006379702,0006479383,0006492347,0007100930,0007103921,0007110227,0007116497,...,9500817314,9505111894,9505156022,9505156057,9583004448,958704049X,9721030325,9722105329,9724605574,B00009NDAN
User-ID,,,,,,,,,,,,,,,,,,,,,
254.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
882.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1435.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1733.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1903.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
user_item.shape

(1085, 4576)

In [23]:
tf_sim = skr.transformer.SimilarityTransformer(cols=(2, -1), index_col='ISBN', normalize=True)
sim_mat = tf_sim.transform(df_sample)

In [24]:
sim_mat.head()

ISBN,0880291540,0385509510,0061092606,0446611859,0818405422,0961035269,0553560735,0074464159,0553251031,0142001740,...,0380816059,0446364703,0933598491,8483101610,0448421763,0856487457,1841491462,0843108371,0449236951,0312932782
ISBN,,,,,,,,,,,,,,,,,,,,,
0880291540,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0385509510,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0061092606,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0446611859,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0818405422,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
sim_mat.shape

(5000, 5000)

In [26]:
rec = skr.recommender.CrossSimilarityRecommender(5)
rec.fit((user_item, sim_mat, ))
rec.predict([10, 12])

ValueError: negative dimensions are not allowed